In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect


In [2]:
csv_file = "data-csv/goals.csv"
goals = pd.read_csv(csv_file)
goals.head()

,key_id,goal_id,tournament_id,tournament_name,match_id,match_name,match_date,stage_name,group_name,team_id,...,shirt_number,player_team_id,player_team_name,player_team_code,minute_label,minute_regulation,minute_stoppage,match_period,own_goal,penalty
0,1,G-0001,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,...,0,T-28,France,FRA,19',19,0,first half,0,0
1,2,G-0002,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,...,0,T-28,France,FRA,40',40,0,first half,0,0
2,3,G-0003,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,...,0,T-28,France,FRA,43',43,0,first half,0,0
3,4,G-0004,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,...,0,T-28,France,FRA,87',87,0,second half,0,0
4,5,G-0005,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-44,...,0,T-44,Mexico,MEX,70',70,0,second half,0,0


In [3]:
new_goals = goals[['goal_id', 'tournament_id','match_id', 'team_id','player_id','minute_label','minute_regulation','minute_stoppage','match_period','own_goal','penalty']].copy()

In [4]:
new_goals['year']= new_goals['tournament_id'].str.strip().str[-4:]
new_goals = new_goals.loc[(new_goals['year'] > '1997')]
new_goals.head()

,goal_id,tournament_id,match_id,team_id,player_id,minute_label,minute_regulation,minute_stoppage,match_period,own_goal,penalty,year
1584,G-1585,WC-1998,M-1998-01,T-09,P-05926,5',5,0,first half,0,0,1998
1585,G-1586,WC-1998,M-1998-01,T-61,P-05404,38',38,0,first half,0,1,1998
1586,G-1587,WC-1998,M-1998-01,T-09,P-04230,74',74,0,second half,1,0,1998
1587,G-1588,WC-1998,M-1998-02,T-45,P-02981,37',37,0,first half,0,0,1998
1588,G-1589,WC-1998,M-1998-02,T-45,P-02819,60',60,0,second half,0,0,1998


In [5]:
#Connect to local database

protocol = 'postgresql'
username = 'postgres'
password = '20168_AUsc'
host = 'localhost'
port = 5432
database_name = 'fifawc_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

In [6]:
insp.get_table_names() 

['tournaments',
 'team_appearances',
 'matches',
 'squads',
 'player_appearances',
 'goals']

In [7]:
# Use pandas to load csv converted DataFrame into database

new_goals.to_sql(name='goals', con=engine, if_exists='append', index=False) 

In [8]:
# Check that data has been fed into table
pd.read_sql_query('select * from goals', con=engine).head()

,goal_id,tournament_id,match_id,team_id,player_id,minute_label,minute_regulation,minute_stoppage,match_period,own_goal,penalty,year
0,G-1585,WC-1998,M-1998-01,T-09,P-05926,5',5,0,first half,0,0,1998
1,G-1586,WC-1998,M-1998-01,T-61,P-05404,38',38,0,first half,0,1,1998
2,G-1587,WC-1998,M-1998-01,T-09,P-04230,74',74,0,second half,1,0,1998
3,G-1588,WC-1998,M-1998-02,T-45,P-02981,37',37,0,first half,0,0,1998
4,G-1589,WC-1998,M-1998-02,T-45,P-02819,60',60,0,second half,0,0,1998


In [9]:
# Join goals with squad to obtain a table that records player information such as position.

sql_join = r"""SELECT * FROM goals
JOIN squads
ON goals.player_id = squads.player_id"""
pd.read_sql_query(sql_join, con=engine).head()

,goal_id,tournament_id,match_id,team_id,player_id,minute_label,minute_regulation,minute_stoppage,match_period,own_goal,...,tournament_id,team_id,team_name,player_id,family_name,given_name,shirt_number,position_name,position_code,year
0,G-1585,WC-1998,M-1998-01,T-09,P-05926,5',5,0,first half,0,...,WC-1998,T-09,Brazil,P-05926,Sampaio,César,5,midfielder,MF,1998
1,G-1586,WC-1998,M-1998-01,T-61,P-05404,38',38,0,first half,0,...,WC-1998,T-61,Scotland,P-05404,Collins,John,11,midfielder,MF,1998
2,G-1587,WC-1998,M-1998-01,T-09,P-04230,74',74,0,second half,1,...,WC-1998,T-61,Scotland,P-04230,Boyd,Tom,3,defender,DF,1998
3,G-1588,WC-1998,M-1998-02,T-45,P-02981,37',37,0,first half,0,...,WC-1998,T-45,Morocco,P-02981,Hadji,Mustapha,7,midfielder,MF,1998
4,G-1589,WC-1998,M-1998-02,T-45,P-02819,60',60,0,second half,0,...,WC-1998,T-45,Morocco,P-02819,Hadda,Abdeljalil,9,forward,FW,1998
